# FASTQ, Phred e QC di base

Obiettivi:
- Capire la struttura del formato FASTQ (4 righe per read).
- Convertire caratteri ASCII in Phred score.
- Calcolare statistiche di qualità e fare un trimming semplice.

In questo notebook usiamo un piccolo FASTQ **sintetico** incluso come stringa.

In [ ]:
from __future__ import annotations
from collections import Counter, defaultdict
import math
import statistics
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

## Dati di esempio

Questo è un piccolo FASTQ con 5 read fittizi (sequenze e qualità inventate).

In [ ]:
fastq_text = """@READ_1
ACGTACGTACGTACGTACGT
+
IIIIHHHHFFFFEEEEDDDD
@READ_2
ACGTNACGTNACGTNACGTN
+
IIIIIIIIHHHHFFFFEEEE
@READ_3
GGGGCCCCAAAATTTTNNNN
+
IIIIIIIIIIIIHHHHFFFF
@READ_4
ACACACACACACACACACAC
+
DDDDFFFFHHHHIIIIIIII
@READ_5
TTTTAAAACCCCGGGGNNNN
+
!!''((((****++++IIII
"""

## Funzioni per leggere il FASTQ e gestire i Phred score

In [ ]:
def phred_char_to_q(ch: str) -> int:
    """Converte un carattere ASCII (Phred+33) in qualità Q intero."""
    return ord(ch) - 33

def q_to_perror(q: int) -> float:
    """Converte un Phred score Q in probabilità di errore P(error)."""
    return 10 ** (-q / 10)

def parse_fastq_from_string(text: str):
    """Generator che restituisce (header, seq, qual) dal testo FASTQ."""
    lines = [l.strip() for l in text.strip().splitlines()]
    for i in range(0, len(lines), 4):
        header = lines[i]
        seq = lines[i + 1]
        plus = lines[i + 2]
        qual = lines[i + 3]
        yield header, seq, qual

reads = list(parse_fastq_from_string(fastq_text))
print(f"Numero di read: {len(reads)}")
reads[0]

## Esploriamo il primo read

In [ ]:
h, s, q = reads[0]
print("Header :", h)
print("Seq    :", s)
print("Qual   :", q)
print("Lunghezza seq:", len(s), " Lunghezza qual:", len(q))

q_scores = [phred_char_to_q(ch) for ch in q]
list(zip(s, q_scores))

Ogni carattere nella quality line rappresenta un Phred score.
Ricordiamo che:

$$Q = -10 \\log_{10}(P_{\\text{errore}})$$

Vediamo la tabellina per alcuni valori tipici.

In [ ]:
for Q in [10, 20, 30, 40]:
    perror = q_to_perror(Q)
    print(
        f"Q={Q:2d}  ->  P_errore={perror:.4f}  ->  accuratezza={(1 - perror) * 100:.3f}%"
    )

## Distribuzione globale dei Phred score nei nostri read

In [ ]:
all_q = []
for _, seq, qual in reads:
    all_q.extend(phred_char_to_q(ch) for ch in qual)

counter = Counter(all_q)
print("Conteggio Phred score:", counter)

qs, counts = zip(*sorted(counter.items()))
plt.bar(qs, counts)
plt.xlabel("Phred Q")
plt.ylabel("Conteggio basi")
plt.title("Distribuzione dei Phred score (dataset sintetico)")
plt.show()

## Qualità media per read

In [ ]:
def mean_q(qual: str) -> float:
    scores = [phred_char_to_q(ch) for ch in qual]
    return statistics.mean(scores)

for header, seq, qual in reads:
    print(header, "Q medio:", round(mean_q(qual), 2))

## QC per posizione: qualità media lungo il read

Calcoliamo la qualità media per posizione (base 0, 1, 2, ...) aggregando tutti i read.

In [ ]:
pos_qualities = defaultdict(list)

for _, seq, qual in reads:
    for i, ch in enumerate(qual):
        pos_qualities[i].append(phred_char_to_q(ch))

positions = sorted(pos_qualities.keys())
mean_q_per_pos = [statistics.mean(pos_qualities[i]) for i in positions]

plt.plot(positions, mean_q_per_pos, marker="o")
plt.xlabel("Posizione nel read (0-based)")
plt.ylabel("Qualità media (Q)")
plt.title("Qualità media per posizione")
plt.show()

In dataset reali il profilo spesso cala verso la fine del read.

## Trimming semplice basato sulla qualità di coda

Implementiamo un trimming banale:
- finché la coda del read ha Q < soglia, tagliamo quella base
- fermandoci quando incontriamo una base con Q >= soglia.

In [ ]:
def trim_read(seq: str, qual: str, min_q: int = 20) -> tuple[str, str]:
    q_scores = [phred_char_to_q(ch) for ch in qual]
    cut_pos = len(q_scores)
    while cut_pos > 0 and q_scores[cut_pos - 1] < min_q:
        cut_pos -= 1
    return seq[:cut_pos], qual[:cut_pos]

min_q_threshold = 20

for header, seq, qual in reads:
    trimmed_seq, trimmed_qual = trim_read(seq, qual, min_q=min_q_threshold)
    print(header)
    print("  original length:", len(seq), "Q medio:", round(mean_q(qual), 2))
    if trimmed_seq:
        print(
            "  trimmed  length:",
            len(trimmed_seq),
            "Q medio:",
            round(mean_q(trimmed_qual), 2),
        )
    else:
        print("  trimmed  length: 0 (read completamente scartato)")
    print()

In un contesto reale:
- il trimming migliora la qualità media delle basi mantenute
- ma riduce la lunghezza del read (a volte fino a scartare il read)

Questo collega direttamente alle scelte di trimming discusse a lezione.